In [4]:
using ControlSystemIdentification, CSV, Random, Plots, DataFrames, Optim

In [2]:
data = CSV.read("data/bmx160_stationary_15hr.csv");

## Try ControlSystemIdentification first
#### With this method, either I'm not getting the result I want or I'm not using that result right. In any case the system doesn't look like it should

In [23]:
# Start with just the x coord:
# y, t = data[:, [:gyrox, :dt]]
y = data[:, :gyroz]
# y = Matrix(y)
u = eps() .* randn(size(y)...) # to avoid singularity give a small input
sys = n4sid(y, u, 3)
# sys = pem(y, u, nx = 3)

ControlSystemIdentification.N4SIDResult(ControlSystems.StateSpace{Float64,Array{Float64,2}}
A = 
 0.9999925091392116       0.02066264486955502  -0.00030777956580522493
 0.00034598574532756517   0.1263745472451653    0.40578917814714693   
 5.3785688492604346e-5   -0.4057230539519991    0.07041125940218443   
B = 
 -2.3828304512222834e9 
 -1.043036587391571e10 
 -3.2455616820256035e10
C = 
 -11.284103136815745  2.806529586847635  -0.13676292500953846
D = 
 -1.4369780266069094e10

Sample Time: 1.0 (seconds)
Discrete-time state-space model, [3.759738082345358e-8 -5.671998833454202e-7 -2.6134871448408427e-8; -5.671998833454202e-7 8.556867466103765e-6 3.9436226893039207e-7; -2.6134871448408427e-8 3.943622689303921e-7 2.2683269976272746e-8], [0.00016619688348979886], [-2.49970943340329e-6; 3.771103173875617e-5; 1.7391700842818772e-6], [-0.016601557395804826; 0.14519809846345297; -0.03810302174541365], [1.1434600850942365e-7 -5.7092664466895e-7 3.47199936849745e-8; -5.7092664466895e-7 8.80783

In [ ]:
# # simulation:
function sim_undriven_system(A, C, K, s=1.0; T = 100, x0 = zeros(axes(A, 2)))
    x = x0
    Y = zeros(T)
    
    for i in 1:T
        e = randn()
        Y[i] = C*x + s*e

        x .= A*x + K*e
    end
    return Y
end

Y = sim_undriven_system(sys.sys.A, vec(sys.sys.C)', vec(sys.K), 0.0001, T = 10^8);
# plot(Y[range(1, size(Y, 1), step = 1000), :])
# using ControlSystems
# lsim(sys.sys, u, 1:10^5)

# Looks nothing like what we want, so what's the deal??

#### Note this is only done for 10^6 steps, but I've also done with 10^8 and it still looks way off. It's also totally different each time you run it (which is not how it should be). I also tried with small values of s since that's definitely what's throwing it off

In [ ]:
using AllanDeviations

res = allandev(Y[:, 1], 1.0, frequency = true)
plot(res.tau, res.deviation, scale = :log10)

In [65]:
#= process:
- start with some initial values of (A, C, K, s)
- simulate the system (linear+whitenoise blah blah)
- calculate the allan variance
- use the allan variance to calculate the loss
    - e.g. RMS difference at each point, or pick knots.
- optimize based on loss
    - flatten the matrices into a big vector opt_vars
    - that's the input (x) to the optimization problem

=#

function unpack(x, n)
    # vars always have the form:
    # A - n x n
    # C - 1 x n
    # K - n x 1
    # s - scalar
    s = last(x)
    A = reshape(x[1:n^2], n, n)
    C = x[(n^2 + 1):(n^2 + n)]'
    K = x[(n^2 + n + 1):end-1]
    return A, C, K, s
end

function opt_func(x, actual_dev, T = 10^5, order = 3)
    
    A, C, K, s = unpack(x, order)
    
    Y = sim_undriven_system(A, C, K, s; T = T, x0 = zeros(axes(A, 2)))
    dev = allandev(Y, 0.05).deviation
    
    loss = rms(dev .- actual_dev)
end

order = 3
x0 = randn(order^2 + 2*order + 1)
T = 10^3
dat = data[1:T, :gyroz]
actual_dev = allandev(dat, 0.05)
rms(x) = sqrt(mapreduce(abs2, +, x)) # helper for root mean square

optimize(x->opt_func(x, actual_dev.deviation, T, order), x0)



 * Status: failure (reached maximum number of iterations) (line search failed)

 * Candidate solution
    Minimizer: [3.23e-01, 4.48e-01, 1.04e+00,  ...]
    Minimum:   2.160151e-01

 * Found with
    Algorithm:     Nelder-Mead
    Initial Point: [3.24e-01, 2.76e-01, 7.97e-01,  ...]

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    2984
